In [1]:
import pandas as pd
import sqlite3
from datetime import datetime, timedelta

conn = sqlite3.connect('gugu.db')

# SQL 查询，获取数据
query = '''
    SELECT 
        users.user_id,
        users.nickname, 
        checkin_records.checkin_time,
        assignments.name AS assignment_name
    FROM 
        users
    LEFT JOIN 
        checkin_records 
    ON 
        checkin_records.user_id = users.user_id
    LEFT JOIN
        assignments
    ON
        checkin_records.assignment_id = assignments.id
'''

# 读取 SQL 查询结果到 Pandas DataFrame
df = pd.read_sql_query(query, conn)

# 关闭数据库连接
conn.close()

# 将 checkin_time 转换为 datetime 格式
df['checkin_time'] = pd.to_datetime(df['checkin_time'])

print(df.head())

# 定义日期范围（假设你有一系列日期）
start_date = datetime(2025, 11, 1)
end_date = datetime(2025, 11,30)
dates = pd.date_range(start_date, end_date)

# 初始化一个空的结果 DataFrame，行是日期，列是用户的昵称
user_nicknames = df['nickname'].unique()
result_df = pd.DataFrame(index=user_nicknames, columns=dates.date)

# 设置时间延迟参数
# time_delay = 16  # 晚于16点的打卡记录算作第二天

# 循环每个用户和日期进行打卡查询和标记
for nickname in user_nicknames:
    user_records = df[df['nickname'] == nickname]

    for date in dates:
        # 定义当天的起止时间
        start_time = datetime.combine(date, datetime.min.time())
        end_time = start_time + timedelta(days=1)

        # 查找用户当天的打卡记录
        checkin_record = user_records[
            (user_records['checkin_time'] >= start_time) &
            (user_records['checkin_time'] < end_time)
            ]

        # 默认是无记录的情况
        date_record = ''

        if not checkin_record.empty:
            assignment_name = checkin_record.iloc[0]['assignment_name']  # 获取该用户当天的任务名称
            if "练笔" in assignment_name:
                date_record = "√输出练笔"
            elif "扒文" in assignment_name:
                date_record = "√扒文扒榜"
            elif "请假" in assignment_name:
                date_record = "×请假"
            else:
                date_record = "√其他"

        # 填充结果 DataFrame 中对应的值
        result_df.loc[nickname,date.date()] = date_record

print(result_df.head())  # 查看结果 DataFrame 的前几行
result_df.to_csv('gugu.csv',encoding='utf-8-sig')

                            user_id nickname               checkin_time  \
0  76D7B4609387188C82BB689F12FE3C66       花花 2025-05-26 21:53:08.521854   
1  76D7B4609387188C82BB689F12FE3C66       花花 2025-05-27 23:10:08.298099   
2  76D7B4609387188C82BB689F12FE3C66       花花 2025-07-19 18:40:45.536395   
3  76D7B4609387188C82BB689F12FE3C66       花花 2025-07-24 15:49:29.986047   
4  76D7B4609387188C82BB689F12FE3C66       花花 2025-07-30 21:06:22.118494   

  assignment_name  
0            输出练笔  
1            输出练笔  
2            输出练笔  
3            输出练笔  
4            输出练笔  
   2025-11-01 2025-11-02 2025-11-03 2025-11-04 2025-11-05 2025-11-06  \
花花                              √其他        √其他        √其他        √其他   
梓涵        √其他                   √其他        √其他        √其他      √扒文扒榜   
娓娓                   √其他        √其他        ×请假        √其他        √其他   
晚夏                              √其他        ×请假                   √其他   
米娅        √其他                   ×请假        √其他        √其他            